In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
os.environ["EnironementMLFLOW_TRACKING_URI"]="https://dagshub.com/mr1aarb23/Teraflow-Forecast-Pipeline.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mr1aarb23"
os.environ["MLFLOW_TRACKING_PASSWORD"]="75375de53126c0efd2af4abfd59e8ab3eb6a0594"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig_LR:
    root_dir : Path
    model_dir_name:str
    X_test_data_path : Path
    y_test_data_path : Path
    model_path : Path
    metric_file : Path
    mlflow_uri : str

In [7]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories,save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config_LR(self) -> ModelEvaluationConfig_LR:
        config = self.config.model_evaluation_LR

        create_directories([config.root_dir])
        create_directories([os.path.join(config.root_dir,config.model_dir_name)])
        
        model_evaluation_config = ModelEvaluationConfig_LR(
            root_dir=config.root_dir,
            model_dir_name=config.model_dir_name,
            X_test_data_path=config.X_test_data_path,
            y_test_data_path=config.y_test_data_path,
            model_path=config.model_path,
            metric_file= config.metric_file,
            mlflow_uri="https://dagshub.com/mr1aarb23/Teraflow-Forecast-Pipeline.mlflow" 
        )
        
        return model_evaluation_config

In [9]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import os
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,mean_absolute_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import joblib
import numpy as np

In [10]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig_LR):
        self.config = config
        
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mape = mean_absolute_percentage_error(actual,pred)
        mae = mean_absolute_error(actual,pred)
        mse = mean_squared_error(actual,pred)
        
        return rmse,mape,mae,mse
    
    def log_into_mlflow(self):
        y_test_data = np.array(pd.read_csv(self.config.y_test_data_path))
        
        X_test_data = np.array(pd.read_csv(self.config.X_test_data_path))
        
        model = joblib.load(self.config.model_path)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            forecast = model.predict(X_test_data)
            (rmse,mape,mae,mse) = self.eval_metrics(y_test_data,forecast)
            
            scores = {"rmse":rmse,"mape":mape,"mae":mae,"mse":mse}
            save_json(path=Path(self.config.metric_file),data=scores)
                        
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mape",mape)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("mse",mse)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="Linear Regression")
            else:
                mlflow.sklearn.log_model(model, "model")
                
            

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config_LR()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-04-18 15:34:12,059: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-18 15:34:12,067: INFO: common: created directory at: artifacts]
[2024-04-18 15:34:12,071: INFO: common: created directory at: artifacts/model_evaluation]
[2024-04-18 15:34:12,074: INFO: common: created directory at: artifacts/model_evaluation\LR]


c:\Users\hp\anaconda3\envs\mlflowEnv\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.1.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2024-04-18 15:34:12,885: INFO: common: json file saved at: artifacts\model_evaluation\LR\lr_metrics.json]
